<a href="https://colab.research.google.com/github/Kriti1400/Implexis-AI/blob/main/support_vector_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from matplotlib import pyplot as plt
%matplotlib inline

import torch
import numpy as np


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.9 MB/s 
     |████████████████████████████████| 182 kB 51.2 MB/s 


In [ ]:
df = pd.read_csv('liberal_conservative_Reddit.csv')
# df = df.loc[8300: 8400]
#keep the first two columns only
df = df.drop(columns = ['Score' , 'Id' , 'Subreddit' , 'URL', 'Num of Comments', 'Text', 'Date Created'])
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

df.head()  #Displaying first five rows of the dataset

Number of training sentences: 12,854



,Title,Political Lean
0,"No matter who someone is, how they look like, ...",Liberal
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal
2,State of the union,Liberal
3,We Should Just Give Poor People Money,Liberal
4,Do it for the Dew,Liberal


In [ ]:
for i in range(len(df['Political Lean'])):
  df['Political Lean'] = df['Political Lean'].replace(to_replace=['Liberal', 'Conservative'], value=[0, 1])

df.head()

,Title,Political Lean
0,"No matter who someone is, how they look like, ...",0
1,Biden speech draws 38.2 million U.S. TV viewers,0
2,State of the union,0
3,We Should Just Give Poor People Money,0
4,Do it for the Dew,0


In [ ]:
titles = df['Title'].values
labels = df['Political Lean'].values

In [ ]:
# df1 = df['Political Lean'] = pd.to_numeric(df['Political Lean'], errors='coerce')
# df.dtypes

# df1 = 'Political Lean'
# df2 = 'Political Lean'

In [ ]:
#plt.xlabel('radius_mean')
#plt.ylabel('texture_mean')
#plt.scatter(df1['radius_mean'], df1['texture_mean'], color='green', marker='+')
#plt.scatter(df2['radius_mean'], df2['texture_mean'], color='blue', marker='.')

In [ ]:
model = SVC(kernel='rbf')

In [ ]:
# Tokenization

from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Print the original title.
print(' Original: ', titles[0])

# Print the title split into tokens.
print('Tokenized: ', tokenizer.tokenize(titles[0]))

# Print the title mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(titles[0])))

## NOTE: still need to check what is happening with words like "todaaay"

 Original:  No matter who someone is, how they look like, what language they speak, what they wear, remember the human. For the sake of humanity, the working class can and must unite across all arbitrary boundaries.
Tokenized:  ['no', 'matter', 'who', 'someone', 'is', ',', 'how', 'they', 'look', 'like', ',', 'what', 'language', 'they', 'speak', ',', 'what', 'they', 'wear', ',', 'remember', 'the', 'human', '.', 'for', 'the', 'sake', 'of', 'humanity', ',', 'the', 'working', 'class', 'can', 'and', 'must', 'unite', 'across', 'all', 'arbitrary', 'boundaries', '.']
Token IDs:  [2053, 3043, 2040, 2619, 2003, 1010, 2129, 2027, 2298, 2066, 1010, 2054, 2653, 2027, 3713, 1010, 2054, 2027, 4929, 1010, 3342, 1996, 2529, 1012, 2005, 1996, 8739, 1997, 8438, 1010, 1996, 2551, 2465, 2064, 1998, 2442, 15908, 2408, 2035, 15275, 7372, 1012]


In [ ]:
max_len = 0

# For every title...
for sent in titles:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum title length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  105


In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = [] 

# For every sentence...
for sent in titles:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens. Attention masks basically pad shorter sentence to match the lenght of the longest sequence
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)     # tokens have been mapped to ids so this is now int
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
# input_ids is a (N x L) tensor, N = number of samples (~12k), L = max sentence length (128)
print('Original: ', titles[0])
print('Token IDs:', input_ids[0])
print('Token IDs:', labels[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  No matter who someone is, how they look like, what language they speak, what they wear, remember the human. For the sake of humanity, the working class can and must unite across all arbitrary boundaries.
Token IDs: tensor([  101,  2053,  3043,  2040,  2619,  2003,  1010,  2129,  2027,  2298,
         2066,  1010,  2054,  2653,  2027,  3713,  1010,  2054,  2027,  4929,
         1010,  3342,  1996,  2529,  1012,  2005,  1996,  8739,  1997,  8438,
         1010,  1996,  2551,  2465,  2064,  1998,  2442, 15908,  2408,  2035,
        15275,  7372,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,  

In [ ]:
x = input_ids
y = labels
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y, test_size=0.2, random_state=True)

print(x_train)
print(y_train)

tensor([[  101,  7177,  1997,  ...,     0,     0,     0],
        [  101,  2515,  4419,  ...,     0,     0,     0],
        [  101,  2231,  1999,  ...,     0,     0,     0],
        ...,
        [  101,  2129, 16824,  ...,     0,     0,     0],
        [  101,  2309,  3477,  ...,     0,     0,     0],
        [  101, 22028,  1010,  ...,     0,     0,     0]])
tensor([0, 0, 1,  ..., 0, 0, 0])


In [ ]:
#import string
#string.punctuation
#df['Title'] = df['Title'].replace('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~', '')


model.fit(x_train, y_train)

SVC()

In [ ]:
predictions = model.predict(x_test)
print(predictions)

[0 0 0 ... 0 0 0]


In [ ]:
percentage = model.score(x_test, y_test)
print(percentage)

0.6483858420847919


In [ ]:
from sklearn.metrics import confusion_matrix
res = confusion_matrix(y_test, predictions)
print("Confusion Matrix")
print(res)
print(f"Test Set: {len(x_test)}")
print(f"Accuracy = {percentage*100} %")

Confusion Matrix
[[1659    5]
 [ 899    8]]
Test Set: 2571
Accuracy = 64.83858420847919 %


In [ ]:
from sklearn.metrics import matthews_corrcoef

y_pred = model.predict(x_test)
y_true = y_test
mcc = matthews_corrcoef(y_true, y_pred)

print(mcc)

0.039178143049511074
